In [ ]:
Sam Swain - Video Games
Last time I ate sushi was at the end of 2020

## Workshop - Regression-Based Classification

Does `statsmodels` marginal effect use the average of covariates or the average predicted values? 
- Use the class data.
- Show your work.

Load the necessary packages and data:

In [17]:
import numpy as np
import pandas as pd
from tqdm import tqdm # progress bar

import statsmodels.api as sm
from sklearn import linear_model as lm

from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc = {'axes.titlesize': 24,
             'axes.labelsize': 20,
             'xtick.labelsize': 12,
             'ytick.labelsize': 12,
             'figure.figsize': (8, 4.5)})
sns.set_style("white")

df = pd.read_csv('class_data.csv')

df_prepped = df.drop(columns = ['urate_bin', 'year']).join([
    pd.get_dummies(df['urate_bin'], drop_first = True),
    pd.get_dummies(df.year, drop_first = True)    
])
df_prepped = df_prepped.drop('GeoName', axis = 1)

Fit a logistic regression using either `sm.Logit()` or `smf.logit()`.

In [20]:
y = df_prepped['pos_net_jobs'].astype(float)
x = df_prepped.drop(columns = 'pos_net_jobs')

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 2/3, random_state = 490)

x_train_std = x_train.apply(lambda x: (x - np.mean(x))/np.std(x), axis = 0)
x_test_std  = x_test.apply(lambda x: (x - np.mean(x))/np.std(x), axis = 0)

fit_logit = sm.Logit(y_train, x_train).fit()
fit_logit.summary2()

Optimization terminated successfully.
         Current function value: 0.602386
         Iterations 6


<class 'statsmodels.iolib.summary2.Summary'>
"""
                          Results: Logit
==================================================================
Model:               Logit            Pseudo R-squared: 0.122     
Dependent Variable:  pos_net_jobs     AIC:              40884.4964
Date:                2021-03-02 15:00 BIC:              41120.5601
No. Observations:    33889            Log-Likelihood:   -20414.   
Df Model:            27               LL-Null:          -23242.   
Df Residuals:        33861            LLR p-value:      0.0000    
Converged:           1.0000           Scale:            1.0000    
No. Iterations:      6.0000                                       
------------------------------------------------------------------
                   Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
------------------------------------------------------------------
fips              -0.0000   0.0000  -1.9504 0.0511 -0.0000  0.0000
pct_d_rgdp         0.0159   0.0014  11.1596 0.0000  0.0131  0.0187
emp_estabs         0.0242   0.0026   9.1429 0.0000  0.0190  0.0294
estabs_entry_rate  0.1724   0.0051  33.7104 0.0000  0.1624  0.1824
estabs_exit_rate  -0.1885   0.0057 -32.9289 0.0000 -0.1997 -0.1773
pop                0.0000   0.0000   4.8203 0.0000  0.0000  0.0000
pop_pct_black     -0.0076   0.0009  -8.5908 0.0000 -0.0094 -0.0059
pop_pct_hisp       0.0057   0.0010   5.8290 0.0000  0.0038  0.0076
lfpr              -0.0120   0.0009 -13.1083 0.0000 -0.0138 -0.0102
density            0.0000   0.0000   1.4658 0.1427 -0.0000  0.0000
lower              0.4354   0.0329  13.2538 0.0000  0.3710  0.4998
similar            0.1906   0.0356   5.3539 0.0000  0.1209  0.2604
2003               0.8198   0.0650  12.6110 0.0000  0.6924  0.9472
2004               0.9919   0.0663  14.9686 0.0000  0.8620  1.1218
2005               1.0553   0.0691  15.2627 0.0000  0.9198  1.1908
2006               1.5792   0.0738  21.4102 0.0000  1.4347  1.7238
2007               0.3139   0.0643   4.8802 0.0000  0.1879  0.4400
2008               0.6296   0.0651   9.6690 0.0000  0.5019  0.7572
2009              -0.8494   0.0776 -10.9462 0.0000 -1.0015 -0.6973
2010              -0.1482   0.0659  -2.2483 0.0246 -0.2775 -0.0190
2011               0.9169   0.0641  14.3112 0.0000  0.7913  1.0425
2012               1.2511   0.0663  18.8804 0.0000  1.1212  1.3810
2013               0.8663   0.0644  13.4473 0.0000  0.7400  0.9926
2014               1.1663   0.0656  17.7914 0.0000  1.0378  1.2948
2015               1.2763   0.0679  18.7901 0.0000  1.1432  1.4094
2016               0.9272   0.0654  14.1664 0.0000  0.7989  1.0554
2017               0.7617   0.0646  11.7916 0.0000  0.6351  0.8883
2018               1.1173   0.0668  16.7383 0.0000  0.9865  1.2482
==================================================================

"""

Get the marginal effects (`.get_margeff()`). Print the summary (`.summary()`).

,fips,GeoName,pct_d_rgdp,emp_estabs,estabs_entry_rate,estabs_exit_rate,pop,pop_pct_black,pop_pct_hisp,lfpr,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
25264,29073,"Gasconade, MO",5.596940,11.358442,6.744,7.004,14746.0,0.942629,1.349519,87.686185,...,0,0,0,0,0,0,0,1,0,0
43044,48189,"Hale, TX",9.365403,17.140449,10.585,12.256,35598.0,5.963818,49.514018,77.452148,...,0,0,0,0,0,0,0,0,0,0
8618,13281,"Towns, GA",-4.047910,9.767857,10.122,14.660,10535.0,0.588514,2.050308,72.434507,...,0,1,0,0,0,0,0,0,0,0
25510,29103,"Knox, MO",1.859859,6.711538,14.563,12.621,4152.0,0.602119,0.818882,96.899545,...,0,0,0,0,0,0,0,0,0,0
44996,48441,"Taylor, TX",2.728216,16.216269,9.781,11.731,125920.0,7.411849,18.904066,77.851508,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33185,38047,"Logan, ND",21.356868,7.508475,10.345,6.897,1927.0,0.518941,1.245459,91.049086,...,0,0,0,0,0,0,1,0,0,0
44320,48355,"Nueces, TX",-6.825017,16.864460,9.849,9.229,325515.0,4.346958,58.515276,74.193686,...,0,0,0,0,0,0,0,0,0,0
43312,48221,"Hood, TX",21.377855,9.332004,16.980,13.015,47952.0,0.613113,9.079913,78.443747,...,0,0,0,0,0,0,0,0,0,0
41410,47171,"Unicoi, TN",22.370500,16.513725,10.317,7.937,17936.0,0.206289,2.899197,71.839986,...,0,0,0,0,0,0,0,0,0,0


***
# Covariate Averages
$$
\frac{\partial p(x_i)}{\partial \beta_1} \approx \frac{e^{\hat{\beta}_0 + \bar{x}\hat{\beta}_1 + \bar{x}\hat{\beta_2}}}{(1 + e^{\hat{\beta}_0 + \bar{x}\hat{\beta}_1 + \bar{x}\hat{\beta_2}})^2}\hat{\beta}
$$

***
# Predicted values Averages
$$
\frac{\partial p(x_i)}{\partial \beta_1} \approx \frac{1}{n} \sum_{i=1}
^n \frac{e^{\hat{y}_i}}{1 + e^{\hat{y}_i}}\hat{\beta}
$$

*** 
# Interpretaton

Interpret the marginal effect on one feature.